In [5]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import bs4
import re
from multiprocessing import Pool, Lock, Value
from time import sleep
import os

In [6]:
docnames = []
import functools
import operator

for d in os.listdir('./content/content'):
    listdocs = os.listdir('./content/content' + '/' + d)
    listdocs = [d + '/' + doc_name for doc_name in listdocs]
    docnames.append(listdocs)
docnames = sorted(functools.reduce(operator.iconcat, docnames, []))
print(docnames[:2])

['20170702/doc.0000.dat', '20170702/doc.0001.dat']


In [7]:
urls = pd.read_csv('./urls.numerate.txt', sep='\t', header=None)
urls.index = urls[1]

In [8]:
def extract_doc_url(docname):
    with open('./content/content/' + docname, errors='ignore') as read_file:
        html = ' '.join(list(read_file))
        return html.split()[0]

In [9]:
import pymorphy2
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
morph = pymorphy2.MorphAnalyzer()

from bs4 import BeautifulSoup


# некоторые стоп слова за таковые не считаем, на всякий случай
# иначе что отвечать на запрос "ЧТО ГДЕ КОГДА"?
my_stopwords = list(set(stopwords.words('russian')) - set(['как', 'когда', 'почему', 'зачем', 'чтобы', 'что']))
my_stopwords[:5]

['там', 'было', 'их', 'вдруг', 'была']

In [10]:
# Кеширование очень сильно ускоряет парсинг (примерно в два раза)
PYMORPHY_CACHE = {}
def lemmatizer(words):
    global PYMORPHY_CACHE
    for word in words:
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = morph.parse(word)[0].normal_form
        yield PYMORPHY_CACHE[word_hash]

In [11]:
import re
from bs4.element import Comment


pattern = re.compile(r"[\W+]")

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(soup):
    #soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)

def clear_html(soup):
    """
    Возвращает (массив_слов_из_заголовка, массив_слов_из_текста)
    """
    #[s.extract() for s in soup(['style', 'script', '[document]', 'meta'])]

    title = soup.title
    if title is not None:
        title = title.text
        title_words = re.sub(pattern, " ", title.lower()).split()
        #soup.title.extract()
    else:
        title_words = []

    text = text_from_html(soup)
    # только буквы и цифры
    filtered = re.sub(r"[\W+]", " ", text.lower())
    tokens = filtered.split()
    
    #длиннее одного символа
    tokens = list(filter(lambda c: len(c) > 1, tokens))
    
   
    #lemmatize + stopwords
    title_words = [word for word in title_words if word not in my_stopwords and word not in stopwords.words('english')]
    title_words = list(lemmatizer(title_words))
    
    
    words = [word for word in tokens if word not in my_stopwords and word not in stopwords.words('english')]
    words = list(lemmatizer(words))
    
    return (title_words, words)

In [12]:
def extract_html_from_file(file_name):
    """
    Возвращает (doc_id, (массив_слов_из_заголовка, массив_слов_из_текста) )
    """
    
    with open('./content/content/' + file_name, errors='ignore') as read_file:
        html = ' '.join(list(read_file))

    url = html.split()[0]
    soup = BeautifulSoup(' '.join(html.split()[1:]))
    doc_id = urls.at[url, 0]
    
    res = clear_html(soup)
    
    
    with open('text-relevance-parsed/content/content/' + file_name + '.txt', 'w+') as f:
        for loc in res:
            for item in loc:
                f.write("%s\n" % item)
            f.write('\n')
    return

In [13]:
for d in os.listdir('./content/content'):
    directory = 'text-relevance-parsed/content/content/' + d
    if not os.path.exists(directory):
        os.makedirs(directory)

In [16]:
from tqdm.notebook import tqdm

for doc in tqdm(docnames):
    extract_html_from_file(doc)